In [4]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.api as sm
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_percentage_error

## PCA

In [17]:
# Load data
file_path = 'compile_tegal.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Pra-pemrosesan data
X = df.iloc[:, 2:]  # Semua kolom setelah kolom ke-2 adalah fitur (Google Trend index)
y = df['Persentase Kemiskinan']
X = X.fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PCA
pca = PCA(n_components=1)  # Menggunakan 2 komponen utama
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


SVR

In [18]:
# Modeling dengan SVR
svr_model = SVR()
svr_model.fit(X_train_pca, y_train)
svr_pred = svr_model.predict(X_test_pca)

# Evaluasi dengan MAPE
svr_mape = mean_absolute_percentage_error(y_test, svr_pred)
print(f"SVR MAPE: {svr_mape}")

SVR MAPE: 0.09930445377736863


KNN

In [19]:
# Modeling dengan KNN
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_pca, y_train)
knn_pred = knn_model.predict(X_test_pca)

# Evaluasi dengan MAPE
knn_mape = mean_absolute_percentage_error(y_test, knn_pred)
print(f"KNN MAPE: {knn_mape}")


AttributeError: 'NoneType' object has no attribute 'split'

ARIMAX

In [15]:
import statsmodels.api as sm

# ARIMAX model
X_train_arimax = sm.add_constant(X_train_pca)  # Menambahkan konstanta
X_test_arimax = sm.add_constant(X_test_pca)

arimax_model = sm.tsa.ARIMA(endog=y_train, exog=X_train_arimax, order=(1, 0, 0))
arimax_result = arimax_model.fit()

# Prediksi
arimax_pred = arimax_result.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test_arimax)

# Evaluasi dengan MAPE
arimax_mape = mean_absolute_percentage_error(y_test, arimax_pred)
print(f"ARIMAX MAPE: {arimax_mape}")

ValueError: A constant trend was included in the model specification, but the `exog` data already contains a column of constants.

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape input data untuk LSTM [samples, timesteps, features]
X_train_lstm = X_train_pca.reshape((X_train_pca.shape[0], 1, X_train_pca.shape[1]))
X_test_lstm = X_test_pca.reshape((X_test_pca.shape[0], 1, X_test_pca.shape[1]))

# Build LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(1, X_train_pca.shape[1])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mape')

# Fit model
lstm_model.fit(X_train_lstm, y_train, epochs=200, verbose=0)

# Prediksi
lstm_pred = lstm_model.predict(X_test_lstm, verbose=0)

# Evaluasi dengan MAPE
lstm_mape = mean_absolute_percentage_error(y_test, lstm_pred)
print(f"LSTM MAPE: {lstm_mape}")


LSTM MAPE: 0.6362023670573468
